# Study of $q_0$ 

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
# needed imports

Using the likelihood 
$$\mathcal{L}(y|\mu+b,\vec{\chi}) = \frac{e^{-(\mu+b)}(\mu+b)^y}{y!}\frac{e^{-\frac{(\mu-\mu_0)^2}{2\sigma_\mu^2}}}{\sqrt{2\pi\sigma_\mu^2}}\frac{e^{-\frac{(b-b_0)^2}{2\sigma_b^2}}}{\sqrt{2\pi\sigma_b^2}}$$
Which gives (keeping only "interesting terms")
$$\ell(\mu,b) = -(\mu+b)+y\ln(\mu+b) - \frac{1}{2}\left(\frac{\mu-\mu_0}{\sigma_\mu}\right)^2 - \frac{1}{2} \left(\frac{b-b_0}{\sigma_b}\right)^2$$
Studying the case for $\sigma_s\rightarrow\infty$ the third term above becomes zero. From this we can find the MLEs for two cases.

First, finding $\hat\mu$, which is done by setting $\frac{\partial\ell(\mu,b)}{\partial\mu}=0$, this gives us $\hat\mu = y - b_0$. Furthermore we can find $\hat b$ by setting $\frac{\partial\ell(\hat\mu,b)}{\partial b}=0$, giving $\hat b = b_0$

Then for the second case, the null hypothesis we find $\hat{\hat b}$ by setting $\frac{\partial\ell(0,b)}{\partial b}=0$, this yields $\hat{\hat{b}} =\frac{b_0-\sigma_b^2+\sqrt{(b_0-\sigma_b^2)^2+4y\sigma_b^2}}{2}$ 

Using these values for the likelihood ratio test statistics gives us

$$
q_0 = -2[\ell(s=0,\hat{\hat b}) - \ell(\hat s, \hat b)]
$$
Giving 
$$
q_0 = 2(y\ln\frac{y}{\hat{\hat b}} + \hat{\hat b} - y) + \left(\frac{\hat{\hat b} -b_0}{\sigma_b}\right)^2
$$

In [2]:
def log_likelihood(mu, b, y, b_0, db):
    ell = -(mu+b) + y*np.log(mu+b) -0.5*((b-b_0)/db)**2
    return ell

def test_Stat_null(y, b_hhat, b_0, db):
    q_0 = 2*(y*np.log(y/b_hhat)+b_hhat - y) + ((b_hhat-b_0)/db)**2
    return q_0

Using the asympotics approximation of Brazzale et al we start by defining the likelihood root

$$
r(\mu) = sign(\hat\mu-\mu)\sqrt{2(\ell_p(\hat\mu, \hat b) - \ell_p(\mu, \hat{\hat b}))} 
$$
where $\ell_p(\mu) = \ell(\mu,\hat{\hat b})$ is the profile log likelihood. For our purposes we (should) have that
$$
r(0) = \sqrt{q_0}
$$

In [3]:
def likelihood_root(mu, mu_hat, b_hat, b_hhat, y, b_0, db):
    ell_p_hat = log_likelihood(mu_hat, b_hat, y, b_0, db)
    ell_p = log_likelihood(mu, b_hhat, y, b_0, db)
    r = np.sign(mu_hat- mu)*np.sqrt(2*(ell_p_hat-ell_p))
    return r

## **First order approximation**

Furthermore, we have the first order modification of the likelihood root (NB when the dimension of $\mu$ is 1, which is the case here)
\begin{equation}
    r^*(\mu) = r(\mu) + \frac{1}{r(\mu)}\log\left(\frac{q(\mu)}{r(\mu)}\right)
\end{equation}
where
$$
q(\mu) = t(\mu) = \sqrt{j_p(\hat\mu)} (\hat\mu - \mu)
$$
is the Wald statistic, or
$$
q(\mu) = s(\mu) = \sqrt{j_p(\hat\mu)}^{-1}\frac{\partial\ell_p(\mu)}{\partial\mu}
$$
is the score statistic. And where 
$$
j_p(\mu) = -\frac{\partial^2\ell_p(\mu)}{\partial\mu^2}
$$
is the observed information function. 

For our purposes, we have that 
$$
\frac{\partial\ell_p(\mu)}{\partial\mu} = -1 + \frac{y}{\mu + \hat{\hat b}}
$$
and
$$
j_p(\mu) = -\frac{\partial^2\ell_p(\mu)}{\partial\mu^2} = \frac{y}{(\mu + \hat{\hat b})^2}
$$

The first order modification works models that are part the exponential family (or transformation family) making it a good approximation. Which the model we are studying in this notebook (or generally in HEP) is not. Meaning that this is not a good "upgrade" of the asymptotic for our purposes, as we shall soon see

In [4]:
def info_func(mu, b_hatt, y):
    j = y/(mu+b_hatt)**2
    return j

def wald(mu_hat, mu, b_hatt, y, b_hat, db):
    j = info_func(mu_hat, b_hatt, y)
    t = np.sqrt(j)*(mu_hat-mu)*np.sqrt(  np.abs(y/(mu_hat + b_hat)**2 +1/db**2)/np.abs(y/(mu + b_hatt)**2 +1/db**2)   )
    return t

def score(mu_hat, mu, b_hatt, y):
    j = info_func(mu_hat, b_hatt, y)
    s = np.sqrt(1/j)*( -1 + y/(mu+b_hatt))
    return s

def modified_likelihood_root(mu, mu_hat, b_hat, b_hhat, y, b_0, db, doWald=True):
    r_mu = likelihood_root(mu, mu_hat, b_hat, b_hhat, y, b_0, db)
    if doWald == True:
        q_mu = wald(mu_hat, mu, b_hhat, y, b_hat, db)
    else: 
        q_mu = score(mu_hat, mu, b_hhat, y)
    return r_mu + (1 / r_mu) * np.log(q_mu / r_mu)


Now for an example using $y=3$, $b_0=0.78$ and $\sigma_b=0.18$, first we compare that indeed $r(0)=\sqrt q_0$

In [5]:
y_obs = 3
b_0 = 0.78
sigma_b = 0.18

mu_hat = y_obs - b_0
b_hatt = (b_0-sigma_b**2+np.sqrt((b_0-sigma_b**2)**2+4*y_obs*(sigma_b**2)))/2
b_hat = b_0

r = likelihood_root(0, mu_hat, b_hat, b_hatt, y_obs, b_0, sigma_b)

q_0 = test_Stat_null(y_obs, b_hatt, b_0, sigma_b)

print(r)
print(np.sqrt(q_0))

1.8477368150191664
1.8477368150191662


Now we check all the modifications

In [6]:
r_s = modified_likelihood_root(0, mu_hat, b_hat, b_hatt, y_obs, b_0, sigma_b) # Wald
r_ss = modified_likelihood_root(0, mu_hat, b_hat, b_hatt, y_obs, b_0, sigma_b, doWald=False) # Score
r_mp = likelihood_root(0, mu_hat, b_hat, b_hatt, y_obs+0.5, b_0, sigma_b) # Mid-p
r_s_mp = modified_likelihood_root(0, mu_hat, b_hat, b_hatt, y_obs+0.5, b_0, sigma_b)
r_ss_mp = modified_likelihood_root(0, mu_hat, b_hat, b_hatt, y_obs+0.5, b_0, sigma_b, doWald=False)


r_sf = stats.norm.sf(r, loc=0)
r_star_sf = stats.norm.sf(r_s, loc=0)
r_stars_sf = stats.norm.sf(r_ss, loc=0)
r_sf_mp = stats.norm.sf(r_mp, loc=0)
r_star_sf_mp = stats.norm.sf(r_s_mp, loc=0)
r_stars_sf_mp = stats.norm.sf(r_ss_mp, loc=0)



In [7]:
#Making a table of the results...

data_sf = {'True value': [0.0264, 1.94], 'r': [r_sf, r], 'r* Wald': [r_star_sf, r_s], 'r* score': [r_stars_sf, r_ss], 
        'Mid-P r': [r_sf_mp, r_mp], 'Mid-P r* Wald': [r_star_sf_mp, r_s_mp], 'Mid-P r* score': [r_stars_sf_mp, r_ss_mp]}
df= pd.DataFrame(data_sf)
df.index = ['p-value', 'Significance']

df 


,True value,r,r* Wald,r* score,Mid-P r,Mid-P r* Wald,Mid-P r* score
p-value,0.0264,0.032320,0.054249,0.010171,0.015410,0.027993,0.005342
Significance,1.9400,1.847737,1.604981,2.319975,2.159381,1.911152,2.552854


## **Higher order density approximations**

To get better results we need to go to higher order approximations of the asymptotic, which means modifying Eq. (1) wrt to these values for $q$:
$$
q_1 = \frac{|\ell_{;\hat\theta}(\hat\theta) - \ell_{;\hat\theta}(\hat\theta_\psi) \qquad \ell_{b;\hat\theta}(\hat\theta_\psi)|}{|\ell_{\theta;\hat\theta}(\hat\theta)|}\left\{\frac{|j_{\theta\theta}(\hat{\theta})|}{|j_{bb}(\hat{\theta}_\psi)|}\right\}^{1/2}
$$
where $|\cdot|$ denotes the determinant, and we are using the notation
$$
\ell_{;X}(Z) = \frac{\partial\ell(\theta)}{\partial X}\Big|_{\theta=Z}\quad\text{and}\qquad \ell_{Y;X}(Z) = \frac{\partial\ell(\theta)}{\partial Y\partial X^T}\Big|_{\theta=Z}
$$
with $\theta=(\mu,b)$ as a shorthand notation and where $\hat\theta = (\hat\mu,\hat b)$ and $\hat\theta_\psi = (\mu, \hat{\hat b})$



**Alternatively** we could use
$$
q_2 = \frac{|\varphi(\hat\theta) - \varphi(\hat\theta_\psi) \qquad \varphi_{\lambda}(\hat\theta_\psi)|}{|\varphi_{\theta}(\hat\theta)|}\left\{\frac{|j_{\theta\theta}(\hat{\theta})|}{|j_{\lambda\lambda}(\hat{\theta}_\psi)|}\right\}^{1/2}
$$
which uses the canonical parameter, $\varphi$, of the tangent exponential model (TEM). Meaning that we have to transform our likelihood. Which is undesirable?


### **Calculations to get the final result...** $q_1$ Edition
Before anything we have to transform the log-likelihood $\ell(\theta,y)$ to be a function of the MLE's and some ancilliary statistics $\ell(\theta;\hat\theta,a)$. We repeat the log-likelhood
$$
\ell(\theta,y) =  -(\mu+b)+y\ln(\mu+b) -\frac{1}{2}\left(\frac{b-b_0}{\sigma_b}\right)^2
$$
We know from the MLEs that we have
$$
\hat\mu = y-b_0,\quad\text{and}\qquad \hat b =b_0
$$
meaning that we can rewrite
$$
y = \hat\mu+\hat b 
$$
***which is what we expect when $y$ is from a Poisson counting experiment***, additionally we can in this example find the simplest ancilliary statistic choice by setting
$$
a=y-(\hat\mu+\hat b) = 0
$$
Thus we can rewrite the log-likelihood to
$$
\ell(\theta;\hat\theta,a) =  -(\mu+b)+(\hat\mu + \hat b +a)\ln(\mu+b) -\frac{1}{2}\left(\frac{b-\hat{b}}{\sigma_b}\right)^2
$$
or since we know $a$
$$
\boxed{\ell(\theta;\hat\theta,a) =  -(\mu+b)+(\hat\mu + \hat b)\ln(\mu+b) -\frac{1}{2}\left(\frac{b-\hat{b}}{\sigma_b}\right)^2}
$$

#### **Calculating $\ell_{;\hat\theta}(\hat\theta)$**
Starting the calculations... we have first that
$$
\ell_{;\hat\theta}(\hat\theta) = \begin{pmatrix} \frac{\partial\ell(\mu,b)}{\partial\hat\mu}\Big|_{\mu=y-b_0,b=b_0}\\
\frac{\partial\ell(\mu,b)}{\partial\hat b}\Big|_{\mu=y-b_0,b=b_0}\end{pmatrix}
$$
where 
$$
\frac{\partial\ell(\mu,b)}{\partial\hat\mu}\Big|_{\mu=y-b_0,b=b_0} = \frac{\partial}{\partial\hat\mu}\left( -(\mu+b)+(\hat\mu+\hat{b})\ln(\mu+b) -\frac{1}{2}\left(\frac{b-\hat{b}}{\sigma_b}\right)^2\right)\Big|_{\mu=y-b_0,b=b_0} = \log(\mu+b)\Big|_{\mu=y-b_0,b=b_0} = \log y
$$
Similarly for $b$ we have
$$
\frac{\partial\ell(\mu,b)}{\partial\hat b}\Big|_{\mu=y-b_0,b=b_0} = \frac{\partial}{\partial\hat\mu}\left( -(\mu+b)+(\hat\mu+\hat b)\ln(\mu+b) -\frac{1}{2}\left(\frac{b-\hat b}{\sigma_b}\right)^2\right)\Big|_{\mu=y-b_0,b=b_0} = \log(\mu+b) + \frac{b-\hat b}{\sigma_b^2}\Big|_{\mu=y-b_0,b=b_0} = \log(y)
$$
Giving 
$$
\boxed{\ell_{;\hat\theta}(\hat\theta) = \begin{pmatrix} \log y\\ \log y\end{pmatrix}}
$$



#### **Calculating $\ell_{;\hat\theta}(\hat\theta_\psi)$**
Next we have that
$$
\ell_{;\hat\theta}(\hat\theta_\psi) = \begin{pmatrix} \frac{\partial\ell(\mu,b)}{\partial\hat\mu}\Big|_{\mu=\mu,b=\hat{\hat b}}\\
\frac{\partial\ell(\mu,b)}{\partial\hat b}\Big|_{\mu=\mu,b=\hat{\hat b}}\end{pmatrix}
$$
using the previous results we get 
$$
\boxed{\ell_{;\hat\theta}(\hat\theta_\psi) = \begin{pmatrix} \log(\mu+\hat{\hat b})\\  \log(\mu+\hat{\hat b}) +\frac{\hat{\hat b}-\hat{b}}{\sigma_b^2}\end{pmatrix}}
$$



#### **Calculating $\ell_{b;\hat\theta}(\hat\theta_\psi)$**
Next we have that
$$
\ell_{b;\hat\theta}(\hat\theta_\psi) = \begin{pmatrix} \frac{\partial^2\ell(\mu,b)}{\partial b \partial\hat\mu^T}\Big|_{\mu=\mu,b=\hat{\hat b}}\\
\frac{\partial^2\ell(\mu,b)}{\partial b\partial\hat b}\Big|_{\mu=\mu,b=\hat{\hat b}}\end{pmatrix}
$$
Using the previous results, we have that 
$$
\frac{\partial}{\partial b}\frac{\partial\ell(\mu,b)}{\partial\hat\mu}\Big|_{\mu=\mu,b=\hat{\hat b}} = \frac{\partial}{\partial b}\log(\mu+b)\Big|_{\mu=\mu,b=\hat{\hat b}} = \frac{1}{\mu + b}\Big|_{\mu=\mu,b=\hat{\hat b}} = \frac{1}{\mu + \hat{\hat b}}
$$
and
$$
\frac{\partial}{\partial b}\frac{\partial\ell(\mu,b)}{\partial\hat b}\Big|_{\mu=\mu,b=\hat{\hat b}} = \frac{\partial}{\partial b}\log(\mu+b) +\frac{b-\hat b}{\sigma_b^2}\Big|_{\mu=\mu,b=\hat{\hat b}} = \frac{1}{\mu + \hat{\hat b}} + \frac{1}{\sigma_b^2}
$$
giving
$$
\boxed{\ell_{b;\hat\theta}(\hat\theta_\psi) = \begin{pmatrix} \frac{1}{\mu + \hat{\hat b}}\\  \frac{1}{\mu + \hat{\hat b}} + \frac{1}{\sigma_b^2}\end{pmatrix}}
$$

#### **Calculating $\ell_{\theta;\hat\theta}(\hat\theta)$**
Starting the calculations... we have first that
$$
\ell_{\theta;\hat\theta}(\hat\theta) = \begin{pmatrix} \frac{\partial^2\ell(\mu,b)}{\partial \mu \partial\hat\mu}\Big|_{\mu=y-b_0,b=b_0} &
\frac{\partial^2\ell(\mu,b)}{\partial b \partial\hat\mu}\Big|_{\mu=y-b_0,b=b_0}\\
\frac{\partial^2\ell(\mu,b)}{\partial\mu\partial\hat b}\Big|_{\mu=y-b_0,b=b_0} &
\frac{\partial^2\ell(\mu,b)}{\partial b\partial\hat b}\Big|_{\mu=y-b_0,b=b_0}\end{pmatrix}
$$
We almost have all of these terms! Starting by finding the easiest term
$$
\frac{\partial^2\ell(\mu,b)}{\partial \mu \partial\hat b}\Big|_{\mu=y-b_0,b=b_0} = \frac{1}{\mu+b}\Big|_{\mu=y-b_0,b=b_0} = \frac{1}{y}
$$
Next we gotta find
$$
\frac{\partial^2\ell(\mu,b)}{\partial \mu \partial\hat\mu}\Big|_{\mu=y-b_0,b=b_0} = \frac{1}{\mu+b}\Big|_{\mu=y-b_0,b=b_0} = \frac{1}{y}
$$
Such that we have
$$
\boxed{\ell_{\theta;\hat\theta}(\hat\theta) = \begin{pmatrix} \frac{1}{y} &
\frac{1}{y}\\
\frac{1}{y} &
\frac{1}{y} + \frac{1}{\sigma_b^2}\end{pmatrix} }
$$

#### **Jacobian temrs**
Lastly we have the Jacobian terms. Where we can identify that
$$
j_{\theta\theta}(\hat\theta) = -\ell_{\theta;\hat\theta}(\hat\theta) 
$$
giving
$$
\Rightarrow |j_{\theta\theta}(\hat\theta)| = \frac{1}{y\sigma_b^2}
$$

The other term we already have, it just has to be evaluated for other values:
$$
j_{bb}(\hat\theta_\psi) =  -\frac{\partial^2\ell(\mu,b)}{\partial b\partial b^T}\Big|_{\mu=\mu,b=\hat{\hat b}}  = \frac{y}{(\mu+\hat{\hat b})^2} + \frac{1}{\sigma_b^2}
$$

#### **Calculating the determinants**
Lastly comes the tedious part of calculating the determinant of the matrices we will get. First we have
$$
|\ell_{;\hat\theta}(\hat\theta) - \ell_{;\hat\theta}(\hat\theta_\psi) \qquad \ell_{b;\hat\theta}(\hat\theta_\psi)| = 
\left|\begin{vmatrix}\log\left(\frac{y}{\mu+\hat{\hat b}}\right) &
\frac{1}{\mu + \hat{\hat b}}\\  
\log\left(\frac{y}{\mu+\hat{\hat b}}\right) - \frac{\hat{\hat b}-\hat{b}}{\sigma_b^2}&
\frac{1}{\mu + \hat{\hat b}} + \frac{1}{\sigma_b^2} \end{vmatrix}\right| = \left| \frac{\log\left(\frac{y}{\mu+\hat{\hat b}}\right)}{\sigma_b^2} - \frac{(\hat{\hat b}-\hat{b})}{(\mu + \hat{\hat b})\sigma_b^2} \right|
$$
and
$$
|\ell_{\theta;\hat\theta}(\hat\theta)| = \frac{1}{y\sigma_b^2}
$$


#### Putting it all together
We now have a formula for the higher order approximation
$$
q_1 = \left| y\log\left(\frac{y}{\mu+\hat{\hat b}}\right) - y\frac{(\hat{\hat b}-\hat{b})}{\mu + \hat{\hat b}} \right|
\left[\frac{\frac{1}{y\sigma_b^2}}{\frac{y}{(\mu+\hat{\hat b})^2} + \frac{1}{\sigma_b^2}}\right]^{1/2}
$$
So now we can test this result!

In [8]:
def higher_order_terms(mu, b_hat, b_hhat, y, db):
        q = np.abs(np.log(y/(mu+b_hhat))*y -y*(b_hhat-b_hat)/((mu+b_hhat)))
        jacobian = np.sqrt(  np.abs(1/(y*db**2)) / np.abs(1/db**2 + (y/(mu+b_hhat)**2)))
        return q*jacobian

def modified_likelihood_root_HO(mu, mu_hat, b_hat, b_hhat, y, b_0, db):
        r_mu = likelihood_root(mu, mu_hat, b_hat, b_hhat, y, b_0, db)
        q_1 = higher_order_terms(mu, b_hat, b_hhat, y, db)
        return r_mu + (1 / r_mu) * np.log(q_1 / r_mu)

In [9]:
r_s_HO = modified_likelihood_root_HO(0, mu_hat, b_hat, b_hatt, y_obs, b_0, sigma_b)

r_sf_HO = stats.norm.sf(r_s_HO, loc=0)

#Making a table of the results...

data_sf = {'True value': [0.0264, 1.94], 'r': [r_sf, r], 'r*': [r_star_sf, r_s], 'r* H.O.': [r_sf_HO, r_s_HO]}
df= pd.DataFrame(data_sf)
df.index = ['p-value', 'Significance']

df 

,True value,r,r*,r* H.O.
p-value,0.0264,0.032320,0.054249,0.031624
Significance,1.9400,1.847737,1.604981,1.857450


# **Testing the canonical parameter**
As briefly mentioned, there is an alternative way of calculating this, we have
$$
q_2 = \frac{|\varphi(\hat\theta) - \varphi(\hat\theta_\psi) \qquad \varphi_{\lambda}(\hat\theta_\psi)|}{|\varphi_{\theta}(\hat\theta)|}\left\{\frac{|j_{\theta\theta}(\hat{\theta})|}{|j_{\lambda\lambda}(\hat{\theta}_\psi)|}\right\}^{1/2}
$$
where we use the canonical parameter $\varphi$ instead. So now we try to find it!
To recapitulate a bit, we have the transformed log-likelihood
\begin{equation}
\ell(\theta;\hat\theta,a) =  -(\mu+b)+(\hat\mu + \hat b)\ln(\mu+b) -\frac{1}{2}\left(\frac{b-\hat{b}}{\sigma_b}\right)^2
\end{equation}
For higher-order asymptotics, one often rewrites the log-likelihood as
$$
\ell(\theta;\hat\theta,a) =  \varphi(\theta)^Ts(y)-h(\theta)+c(y)
$$
where $\varphi(\theta)$ is the **canonical parameter** that we are interested in, $s(y)$ is a *sufficient statistic*, $h(\theta)$ captures additional terms dependent on $\theta$, and $c(y)$ is a term independent of $\theta$, that usually is ignored in inference. 

Eq(1) suggests that 
$$
\varphi(\mu,b) = \ln(\mu+b)
$$ is a good canonical parameter, since $s(y)=y=\hat\mu+\hat b+ a$ follows this form.

### Finding remaining parts
From this it is trivial to find that
$$
\varphi_\theta(\hat{\theta}) = \begin{pmatrix} \frac{\partial\varphi}{\partial\mu}\Big|_{\theta=\hat\theta}\\
\frac{\partial\varphi}{\partial b}\Big|_{\theta=\hat\theta}\end{pmatrix}
= \begin{pmatrix} \frac{1}{\hat\mu+\hat b}\\\frac{1}{\hat\mu+\hat b}\end{pmatrix}
= \begin{pmatrix} \frac{1}{y}\\\frac{1}{y}\end{pmatrix}
$$
and even easier now to see that
$$
\varphi_\lambda(\hat\theta_\psi) = \frac{1}{\mu+\hat{\hat b}}
$$

### Putting it all togheter
That means that we now have
$$
|\varphi(\hat\theta) - \varphi(\hat\theta_\psi) \qquad \varphi_{\lambda}(\hat\theta_\psi)| = \begin{vmatrix}\log\left(\frac{y}{\mu+\hat{\hat b}}\right)& \frac{1}{\mu+\hat{\hat b}}\end{vmatrix}
$$
were we now note that $|\cdot|$ is no longer the determinant, but rather the Euclidian norm. This yields
$$
|\varphi(\hat\theta) - \varphi(\hat\theta_\psi) \qquad \varphi_{\lambda}(\hat\theta_\psi)| = \sqrt{\log\left(\frac{y}{\mu+\hat{\hat b}}\right)^2 + \frac{1}{(\mu+\hat{\hat b})^2}}
$$
Similarly we have
$$
\varphi_\theta(\hat\theta) = \frac{\sqrt{2}}{y^2}
$$
The fisher information term remains the same as for $q_1$ meaning that we have
$$
q_2 = \frac{y^2\sqrt{\log\left(\frac{y}{\mu+\hat{\hat b}}\right)^2 + \frac{1}{(\mu+\hat{\hat b})^2}}}{\sqrt{2}}\left[\frac{\frac{1}{y\sigma_b^2}}{\frac{y}{(\mu+\hat{\hat b})^2} + \frac{1}{\sigma_b^2}}\right]^{1/2}
$$

In [10]:
def higher_order_terms_2(mu, b_hhat, y, db):
        numerator = y**2*np.sqrt(np.log(y/(mu+b_hhat))**2 + 1/(mu+b_hhat)**2 )
        denominator = np.sqrt(2)
        q = numerator/denominator
        jacobian = np.sqrt(  np.abs(1/(y*db**2)) / np.abs(1/db**2 + (y/(mu+b_hhat)**2)))
        return q*jacobian

def modified_likelihood_root_HO_2(mu, mu_hat, b_hat, b_hhat, y, b_0, db):
        r_mu = likelihood_root(mu, mu_hat, b_hat, b_hhat, y, b_0, db)
        q_2 = higher_order_terms_2(mu, b_hhat, y, db)
        return r_mu + (1 / r_mu) * np.log(q_2 / r_mu)

In [11]:
r_s_HO2 = modified_likelihood_root_HO_2(0, mu_hat, b_hat, b_hatt, y_obs, b_0, sigma_b)

r_sf_HO2 = stats.norm.sf(r_s_HO, loc=0)

#Making a table of the results...

data_sf = {'True value': [0.0264, 1.94],'r': [r_sf, r], 'r*': [r_star_sf, r_s], 'r* H.O.': [r_sf_HO, r_s_HO], 'r* H.O. canon': [r_sf_HO2, r_s_HO2]}
df= pd.DataFrame(data_sf)
df.index = ['p-value', 'Significance']

df 

,True value,r,r*,r* H.O.,r* H.O. canon
p-value,0.0264,0.032320,0.054249,0.031624,0.031624
Significance,1.9400,1.847737,1.604981,1.857450,2.475407


From this we see that using $q_1$ yields a somewhat higher significance than just

# OLD TESTING 

## **Higher order density approximations**  WRONG TRANSFORMATION

To get better results we need to go to higher order approximations of the asymptotic, which means modifying Eq. (1) wrt to these values for $q$:
$$
q_1 = \frac{|\ell_{;\hat\theta}(\hat\theta) - \ell_{;\hat\theta}(\hat\theta_\psi) \qquad \ell_{b;\hat\theta}(\hat\theta_\psi)|}{|\ell_{\theta;\hat\theta}(\hat\theta)|}\left\{\frac{|j_{\theta\theta}(\hat{\theta})|}{|j_{bb}(\hat{\theta}_\psi)|}\right\}^{1/2}
$$
where $|\cdot|$ denotes the determinant, and we are using the notation
$$
\ell_{;X}(Z) = \frac{\partial\ell(\theta)}{\partial X}\Big|_{\theta=Z}\quad\text{and}\qquad \ell_{Y;X}(Z) = \frac{\partial\ell(\theta)}{\partial Y\partial X^T}\Big|_{\theta=Z}
$$
with $\theta=(\mu,b)$ as a shorthand notation and where $\hat\theta = (\hat\mu,\hat b)$ and $\hat\theta_\psi = (\mu, \hat{\hat b})$



**Alternatively** we could use
$$
q_2 = \frac{|\varphi(\hat\theta) - \varphi(\hat\theta_\psi) \qquad \varphi_{\lambda}(\hat\theta_\psi)|}{|\varphi_{\theta}(\hat\theta)|}\left\{\frac{|j_{\theta\theta}(\hat{\theta})|}{|j_{\lambda\lambda}(\hat{\theta}_\psi)|}\right\}^{1/2}
$$
which uses the canonical parameter, $\varphi$, of the tangent exponential model (TEM). Meaning that we have to transform our likelihood. Which is undesirable?


### **Calculations to get the final result...**
#### **Calculating $\ell_{;\hat\theta}(\hat\theta)$**
Starting the calculations... we have first that
$$
\ell_{;\hat\theta}(\hat\theta) = \begin{pmatrix} \frac{\partial\ell(\mu,b)}{\partial\hat\mu}\Big|_{\mu=y-b_0,b=b_0}\\
\frac{\partial\ell(\mu,b)}{\partial\hat b}\Big|_{\mu=y-b_0,b=b_0}\end{pmatrix}
$$
where 
$$
\frac{\partial\ell(\mu,b)}{\partial\hat\mu}\Big|_{\mu=y-b_0,b=b_0} = \frac{\partial}{\partial\hat\mu}\left( -(\mu+b)+y\ln(\mu+b) -\frac{1}{2}\left(\frac{b-b_0}{\sigma_b}\right)^2\right)\Big|_{\mu=y-b_0,b=b_0}
$$
substituting $y=\hat\mu +b_0$ we get
$$
\frac{\partial\ell(\mu,b)}{\partial\hat\mu}\Big|_{\mu=y-b_0,b=b_0} = \frac{\partial}{\partial\hat\mu}\left( -(\mu+b)+(\hat\mu+b_0)\ln(\mu+b) -\frac{1}{2}\left(\frac{b-b_0}{\sigma_b}\right)^2\right)\Big|_{\mu=y-b_0,b=b_0} = \log(\mu+b)\Big|_{\mu=y-b_0,b=b_0} = \log y
$$
Similarly for $\hat b$ we have
$$
\frac{\partial\ell(\mu,b)}{\partial\hat b}\Big|_{\mu=y-b_0,b=b_0} = \frac{\partial}{\partial\hat\mu}\left( -(\mu+b)+y\ln(\mu+b) -\frac{1}{2}\left(\frac{b-\hat b}{\sigma_b}\right)^2\right)\Big|_{\mu=y-b_0,b=b_0} = \frac{b-\hat b}{\sigma_b^2}\Big|_{\mu=y-b_0,b=b_0} = \frac{b_0-b_0}{\sigma_b^2}=0
$$
Giving 
$$
\boxed{\ell_{;\hat\theta}(\hat\theta) = \begin{pmatrix} \log y\\ 0\end{pmatrix}}
$$



#### **Calculating $\ell_{;\hat\theta}(\hat\theta_\psi)$**
Next we have that
$$
\ell_{;\hat\theta}(\hat\theta_\psi) = \begin{pmatrix} \frac{\partial\ell(\mu,b)}{\partial\hat\mu}\Big|_{\mu=\mu,b=\hat{\hat b}}\\
\frac{\partial\ell(\mu,b)}{\partial\hat b}\Big|_{\mu=\mu,b=\hat{\hat b}}\end{pmatrix}
$$
using the previous results we get 
$$
\boxed{\ell_{;\hat\theta}(\hat\theta_\psi) = \begin{pmatrix} \log(\mu+\hat{\hat b})\\  \frac{\hat{\hat b}-\hat{b}}{\sigma_b^2}\end{pmatrix}}
$$



#### **Calculating $\ell_{b;\hat\theta}(\hat\theta_\psi)$**
Next we have that
$$
\ell_{b;\hat\theta}(\hat\theta_\psi) = \begin{pmatrix} \frac{\partial^2\ell(\mu,b)}{\partial b \partial\hat\mu^T}\Big|_{\mu=\mu,b=\hat{\hat b}}\\
\frac{\partial^2\ell(\mu,b)}{\partial b\partial\hat b}\Big|_{\mu=\mu,b=\hat{\hat b}}\end{pmatrix}
$$
Using the previous results, we have that 
$$
\frac{\partial}{\partial b}\frac{\partial\ell(\mu,b)}{\partial\hat\mu}\Big|_{\mu=\mu,b=\hat{\hat b}} = \frac{\partial}{\partial b}\log(\mu+b)\Big|_{\mu=\mu,b=\hat{\hat b}} = \frac{1}{\mu + b}\Big|_{\mu=\mu,b=\hat{\hat b}} = \frac{1}{\mu + \hat{\hat b}}
$$
and
$$
\frac{\partial}{\partial b}\frac{\partial\ell(\mu,b)}{\partial\hat b}\Big|_{\mu=\mu,b=\hat{\hat b}} = \frac{\partial}{\partial b}\frac{b-\hat b}{\sigma_b^2}\Big|_{\mu=\mu,b=\hat{\hat b}} = \frac{1}{\sigma_b^2}
$$
giving
$$
\boxed{\ell_{b;\hat\theta}(\hat\theta_\psi) = \begin{pmatrix} \frac{1}{\mu + \hat{\hat b}}\\  \frac{1}{\sigma_b^2}\end{pmatrix}}
$$

#### **Calculating $\ell_{\theta;\hat\theta}(\hat\theta)$**
Starting the calculations... we have first that
$$
\ell_{\theta;\hat\theta}(\hat\theta) = \begin{pmatrix} \frac{\partial^2\ell(\mu,b)}{\partial \mu \partial\hat\mu}\Big|_{\mu=y-b_0,b=b_0} &
\frac{\partial^2\ell(\mu,b)}{\partial b \partial\hat\mu}\Big|_{\mu=y-b_0,b=b_0}\\
\frac{\partial^2\ell(\mu,b)}{\partial\mu\partial\hat b}\Big|_{\mu=y-b_0,b=b_0} &
\frac{\partial^2\ell(\mu,b)}{\partial b\partial\hat b}\Big|_{\mu=y-b_0,b=b_0}\end{pmatrix}
$$
We almost have all of these terms! Starting by finding the easiest term
$$
\frac{\partial^2\ell(\mu,b)}{\partial \mu \partial\hat b}\Big|_{\mu=y-b_0,b=b_0} = 0
$$
Next we gotta find
$$
\frac{\partial^2\ell(\mu,b)}{\partial \mu \partial\hat\mu}\Big|_{\mu=y-b_0,b=b_0} = \frac{1}{\mu+b}\Big|_{\mu=y-b_0,b=b_0} = \frac{1}{y}
$$
Such that we have
$$
\boxed{\ell_{\theta;\hat\theta}(\hat\theta) = \begin{pmatrix} \frac{1}{y} &
\frac{1}{y}\\
0 &
\frac{1}{\sigma_b^2}\end{pmatrix} }
$$

#### **Jacobian temrs**
Lastly we have the Jacobian terms. Where we can identify that
$$
j_{\theta\theta}(\hat\theta) = -\ell_{\theta;\hat\theta}(\hat\theta) 
$$
giving
$$
\Rightarrow |j_{\theta\theta}(\hat\theta)| = \frac{1}{y\sigma_b^2}
$$

The other term we already have, it just has to be evaluated for other values:
$$
j_{bb}(\hat\theta_\psi) =  -\frac{\partial^2\ell(\mu,b)}{\partial b\partial b^T}\Big|_{\mu=\mu,b=\hat{\hat b}}  = \frac{y}{(\mu+\hat{\hat b})^2} + \frac{1}{\sigma_b^2}
$$

#### **Calculating the determinants**
Lastly comes the tedious part of calculating the determinant of the matrices we will get. First we have
$$
|\ell_{;\hat\theta}(\hat\theta) - \ell_{;\hat\theta}(\hat\theta_\psi) \qquad \ell_{b;\hat\theta}(\hat\theta_\psi)| = 
\left|\begin{vmatrix}\log\left(\frac{y}{\mu+\hat{\hat b}}\right) &
\frac{1}{\mu + \hat{\hat b}}\\  
-\frac{b-\hat{\hat b}}{\sigma_b^2}& 
\frac{1}{\sigma_b^2} \end{vmatrix}\right| = \frac{\log\left(\frac{y}{\mu+\hat{\hat b}}\right)}{\sigma_b^2} + \frac{(b-\hat{\hat b})}{(\mu + \hat{\hat b})\sigma_b^2} 
$$
and
$$
|\ell_{\theta;\hat\theta}(\hat\theta)| = \frac{1}{y\sigma_b^2}
$$


#### Putting it all together
We now have a formula for the higher order approximation
$$
q_1 = \left|-y\left((\hat{\hat b}-\hat{b})\log\left(\frac{y}{\mu+\hat{\hat b}}\right) + \frac{1}{\mu + \hat{\hat b}}\right)\right|\left[\frac{\frac{1}{y\sigma_b^2}}{\frac{y}{(\mu+\hat{\hat b})^2} + \frac{1}{\sigma_b^2}}\right]^{1/2}
$$
So now we can test this result!

In [12]:
def higher_order_terms(mu, b_hat, b_hhat, y, db):
        q = np.abs(-y*( (b_hhat - b_hat)*np.log( y/(mu+b_hhat) ) + 1/(mu+b_hhat) ))
        jacobian = np.sqrt(  (1/(y*db**2)) / (1/db**2 + (y/(mu+b_hhat)**2)))
        return q*jacobian

def modified_likelihood_root_HO(mu, mu_hat, b_hat, b_hhat, y, b_0, db):
        r_mu = likelihood_root(mu, mu_hat, b_hat, b_hhat, y, b_0, db)
        q_1 = higher_order_terms(mu, b_hat, b_hhat, y, db)
        return r_mu + (1 / r_mu) * np.log(q_1 / r_mu)

In [13]:
r_s_HO = modified_likelihood_root_HO(0, mu_hat, b_hat, b_hatt, y_obs, b_0, sigma_b)

r_sf_HO = stats.norm.sf(r_s_HO, loc=0)


y_obs_mp = 3.5
mu_hat_mp = y_obs_mp - b_0
b_hatt_mp = (b_0-sigma_b**2+np.sqrt((b_0-sigma_b**2)**2+4*y_obs_mp*(sigma_b**2)))/2

r_s_HO_mp = modified_likelihood_root_HO(0, mu_hat_mp, b_hat, b_hatt_mp, y_obs_mp, b_0, sigma_b)

r_sf_HO_mp = stats.norm.sf(r_s_HO_mp, loc=0)

#Making a table of the results...

data_sf = {'r': [r_sf, r], 'r*': [r_star_sf, r_s], 'r* H.O.': [r_sf_HO, r_s_HO], 'Mid-P r* H.O.': [r_sf_HO_mp, r_s_HO_mp]}
df= pd.DataFrame(data_sf)
df.index = ['p-value', 'Significance']

df 

,r,r*,r* H.O.,Mid-P r* H.O.
p-value,0.032320,0.054249,0.028352,0.014409
Significance,1.847737,1.604981,1.905583,2.185969


## **Higher order density approximations**  WRONG DERIVATIONS (JUST TO TEST)

To get better results we need to go to higher order approximations of the asymptotic, which means modifying Eq. (1) wrt to these values for $q$:
$$
q_1 = \frac{|\ell_{;\hat\theta}(\hat\theta) - \ell_{;\hat\theta}(\hat\theta_\psi) \qquad \ell_{b;\hat\theta}(\hat\theta_\psi)|}{|\ell_{\theta;\hat\theta}(\hat\theta)|}\left\{\frac{|j_{\theta\theta}(\hat{\theta})|}{|j_{bb}(\hat{\theta}_\psi)|}\right\}^{1/2}
$$
where $|\cdot|$ denotes the determinant, and we are using the notation
$$
\ell_{;X}(Z) = \frac{\partial\ell(\theta)}{\partial X}\Big|_{\theta=Z}\quad\text{and}\qquad \ell_{Y;X}(Z) = \frac{\partial\ell(\theta)}{\partial Y\partial X^T}\Big|_{\theta=Z}
$$
with $\theta=(\mu,b)$ as a shorthand notation and where $\hat\theta = (\hat\mu,\hat b)$ and $\hat\theta_\psi = (\mu, \hat{\hat b})$



**Alternatively** we could use
$$
q_2 = \frac{|\varphi(\hat\theta) - \varphi(\hat\theta_\psi) \qquad \varphi_{\lambda}(\hat\theta_\psi)|}{|\varphi_{\theta}(\hat\theta)|}\left\{\frac{|j_{\theta\theta}(\hat{\theta})|}{|j_{\lambda\lambda}(\hat{\theta}_\psi)|}\right\}^{1/2}
$$
which uses the canonical parameter, $\varphi$, of the tangent exponential model (TEM). Meaning that we have to transform our likelihood. Which is undesirable?



### **Calculations to get the final result...**
#### **Calculating $\ell_{;\hat\theta}(\hat\theta)$**
Starting the calculations... we have first that
$$
\ell_{;\hat\theta}(\hat\theta) = \begin{pmatrix} \frac{\partial\ell(\mu,b)}{\partial\mu}\Big|_{\mu=y-b_0,b=b_0}\\
\frac{\partial\ell(\mu,b)}{\partial b}\Big|_{\mu=y-b_0,b=b_0}\end{pmatrix}
$$
where 
$$
\frac{\partial\ell(\mu,b)}{\partial\mu}\Big|_{\mu=y-b_0,b=b_0} = \frac{\partial}{\partial\mu}\left( -(\mu+b)+y\ln(\mu+b) -\frac{1}{2}\left(\frac{b-b_0}{\sigma_b}\right)^2\right)\Big|_{\mu=y-b_0,b=b_0}
$$
giving
$$
-1 +\frac{y}{\hat\mu+\hat b} = 0
$$
Similarly for $\hat b$ we have
$$
\frac{\partial\ell(\mu,b)}{\partial b}\Big|_{\mu=y-b_0,b=b_0} = \frac{\partial}{\partial b}\left( -(\mu+b)+y\ln(\mu+b) -\frac{1}{2}\left(\frac{b-b_0}{\sigma_b}\right)^2\right)\Big|_{\mu=y-b_0,b=b_0} = -1 +\frac{y}{\hat\mu+\hat b} - \frac{\hat b-b_0}{\sigma_b^2}\Big|_{\mu=y-b_0,b=b_0} =0
$$
Giving 
$$
\boxed{\ell_{;\hat\theta}(\hat\theta) = \begin{pmatrix} 0\\ 0\end{pmatrix}}
$$




#### **Calculating $\ell_{;\hat\theta}(\hat\theta_\psi)$**
Next we have that
$$
\ell_{;\hat\theta}(\hat\theta_\psi) = \begin{pmatrix} \frac{\partial\ell(\mu,b)}{\partial\mu}\Big|_{\mu=\mu,b=\hat{\hat b}}\\
\frac{\partial\ell(\mu,b)}{\partial b}\Big|_{\mu=\mu,b=\hat{\hat b}}\end{pmatrix}
$$
using the previous results we get 
$$
\boxed{\ell_{;\hat\theta}(\hat\theta_\psi) = \begin{pmatrix} -1 +\frac{y}{\mu+\hat{\hat b}}\\  -1 +\frac{y}{\hat\mu+\hat{\hat b}} - \frac{\hat{\hat b}-b_0}{\sigma_b^2}\end{pmatrix}}
$$




#### **Calculating $\ell_{b;\hat\theta}(\hat\theta_\psi)$**
Next we have that
$$
\ell_{b;\hat\theta}(\hat\theta_\psi) = \begin{pmatrix} \frac{\partial^2\ell(\mu,b)}{\partial b \partial\mu^T}\Big|_{\mu=\mu,b=\hat{\hat b}}\\
\frac{\partial^2\ell(\mu,b)}{\partial b\partial b^T}\Big|_{\mu=\mu,b=\hat{\hat b}}\end{pmatrix}
$$
Using the previous results, we have that 
$$
\frac{\partial}{\partial b}\frac{\partial\ell(\mu,b)}{\partial\mu}\Big|_{\mu=\mu,b=\hat{\hat b}} = \frac{\partial}{\partial b}\left(-1 +\frac{y}{\mu+b} \right)\Big|_{\mu=\mu,b=\hat{\hat b}} = -\frac{y}{(\mu + b)^2}\Big|_{\mu=\mu,b=\hat{\hat b}} = -\frac{y}{(\mu+\hat{\hat b})^2}
$$
and
$$
\frac{\partial}{\partial b}\frac{\partial\ell(\mu,b)}{\partial b}\Big|_{\mu=\mu,b=\hat{\hat b}} = \frac{\partial}{\partial b}\left(-1 +\frac{y}{\mu+b} - \frac{ b-b_0}{\sigma_b^2}\right)\Big|_{\mu=\mu,b=\hat{\hat b}} = -\frac{y}{(\mu+\hat{\hat b})^2} -\frac{1}{\sigma_b^2}
$$
giving
$$
\boxed{\ell_{b;\hat\theta}(\hat\theta_\psi) = \begin{pmatrix} -\frac{y}{(\mu+\hat{\hat b})^2}\\  -\frac{y}{(\mu+\hat{\hat b})^2} -\frac{1}{\sigma_b^2}\end{pmatrix}}
$$


#### **Calculating $\ell_{\theta;\hat\theta}(\hat\theta)$**
Starting the calculations... we have first that
$$
\ell_{\theta;\hat\theta}(\hat\theta) = \begin{pmatrix} \frac{\partial^2\ell(\mu,b)}{\partial \mu \partial\mu}\Big|_{\mu=y-b_0,b=b_0} &
\frac{\partial^2\ell(\mu,b)}{\partial b \partial\mu}\Big|_{\mu=y-b_0,b=b_0}\\
\frac{\partial^2\ell(\mu,b)}{\partial\mu\partial b}\Big|_{\mu=y-b_0,b=b_0} &
\frac{\partial^2\ell(\mu,b)}{\partial b\partial b}\Big|_{\mu=y-b_0,b=b_0}\end{pmatrix}
$$
We almost have all of these terms! Starting by finding the easiest term
$$
\frac{\partial^2\ell(\mu,b)}{\partial \mu \partial b}\Big|_{\mu=y-b_0,b=b_0} = \frac{\partial}{\partial \mu}\left(-1 +\frac{y}{\mu+b} \right)\Big|_{\mu=\hat\mu,b=\hat{b}} = -\frac{1}{y} 
$$
Next we gotta find
$$
\frac{\partial^2\ell(\mu,b)}{\partial \mu \partial\mu}\Big|_{\mu=y-b_0,b=b_0} = -\frac{y}{(\mu+b)^2}\Big|_{\mu=y-b_0,b=b_0} = -\frac{1}{y}
$$
Such that we have
$$
\boxed{\ell_{\theta;\hat\theta}(\hat\theta) = \begin{pmatrix} -\frac{1}{y} &
-\frac{1}{y}\\
-\frac{1}{y} &
-\frac{1}{y} -\frac{1}{\sigma_b^2}\end{pmatrix} }
$$


#### **Jacobian temrs**
Lastly we have the Jacobian terms. Where we can identify that
$$
j_{\theta\theta}(\hat\theta) = -\ell_{\theta;\hat\theta}(\hat\theta) 
$$
giving
$$
\Rightarrow |j_{\theta\theta}(\hat\theta)| = \frac{1}{y\sigma_b^2}
$$

The other term we already have, it just has to be evaluated for other values:
$$
j_{bb}(\hat\theta_\psi) =  -\frac{\partial^2\ell(\mu,b)}{\partial b\partial b^T}\Big|_{\mu=\mu,b=\hat{\hat b}}  = \frac{y}{(\mu+\hat{\hat b})^2} + \frac{1}{\sigma_b^2}
$$


#### **Calculating the determinants**
Lastly comes the tedious part of calculating the determinant of the matrices we will get. First we have
$$
|\ell_{;\hat\theta}(\hat\theta) - \ell_{;\hat\theta}(\hat\theta_\psi) \qquad \ell_{b;\hat\theta}(\hat\theta_\psi)| = 
\left|\begin{pmatrix} 1 -\frac{y}{\mu+\hat{\hat b}}&
 -\frac{y}{(\mu+\hat{\hat b})^2}\\
1 -\frac{y}{\mu+\hat{\hat b}} &
-\frac{y}{(\mu+\hat{\hat b})^2} -\frac{1}{\sigma_b^2} \end{pmatrix}\right| =\frac{\mu+\hat{\hat b}-y}{(\mu+\hat{\hat b})\sigma_b^2}
$$
and
$$
|\ell_{\theta;\hat\theta}(\hat\theta)| = \frac{1}{y\sigma_b^2}
$$



#### Putting it all together
We now have a formula for the higher order approximation
$$
q_1 = \left|-y\left((\hat{\hat b}-\hat{b})\log\left(\frac{y}{\mu+\hat{\hat b}}\right) + \frac{1}{\mu + \hat{\hat b}}\right)\right|\left[\frac{\frac{1}{y\sigma_b^2}}{\frac{y}{(\mu+\hat{\hat b})^2} + \frac{1}{\sigma_b^2}}\right]^{1/2}
$$
So now we can test this result!


In [14]:
def higher_order_terms(mu, b_hhat, y, db):
        numerator = np.abs((mu+b_hhat-y)/((mu+b_hhat)*db**2))
        denoninator = np.abs(1/(y*db**2))
        q = numerator/denoninator
        print(q)
        jacobian = np.sqrt(  np.abs((1/(y*db**2))) / np.abs(1/db**2 + (y/(mu+b_hhat)**2)))
        return q*jacobian

def modified_likelihood_root_HO(mu, mu_hat, b_hat, b_hhat, y, b_0, db):
        r_mu = likelihood_root(mu, mu_hat, b_hat, b_hhat, y, b_0, db)
        q_1 = higher_order_terms(mu, b_hhat, y, db)
        return r_mu + (1 / r_mu) * np.log(q_1 / r_mu)

r_s_HO = modified_likelihood_root_HO(0, mu_hat, b_hat, b_hatt, y_obs, b_0, sigma_b)

r_sf_HO = stats.norm.sf(r_s_HO, loc=0)


y_obs_mp = 3.5
mu_hat_mp = y_obs_mp - b_0
b_hatt_mp = (b_0-sigma_b**2+np.sqrt((b_0-sigma_b**2)**2+4*y_obs_mp*(sigma_b**2)))/2

r_s_HO_mp = modified_likelihood_root_HO(0, mu_hat_mp, b_hat, b_hatt_mp, y_obs_mp, b_0, sigma_b)

r_sf_HO_mp = stats.norm.sf(r_s_HO_mp, loc=0)

#Making a table of the results...

data_sf = {'r': [r_sf, r], 'r*': [r_star_sf, r_s], 'r* H.O.': [r_sf_HO, r_s_HO], 'Mid-P r* H.O.': [r_sf_HO_mp, r_s_HO_mp]}
df= pd.DataFrame(data_sf)
df.index = ['p-value', 'Significance']

df 

7.458416794992058
10.469399415624396


,r,r*,r* H.O.,Mid-P r* H.O.
p-value,0.032320,0.054249,0.011535,0.004967
Significance,1.847737,1.604981,2.272263,2.578139
